In [5]:
# !pip install simpletransformers

In [1]:
import logging
import os
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from simpletransformers.ner import NERModel, NERArgs

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

random.seed(123)
np.random.seed(456)

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.NOTSET)

In [2]:
train_data = [
    [0, "Harry", "B-PER"],
    [0, "Potter", "I-PER"],
    [0, "was", "O"],
    [0, "a", "O"],
    [0, "student", "B-MISC"],
    [0, "at", "O"],
    [0, "Hogwarts", "B-LOC"],
    [1, "Albus", "B-PER"],
    [1, "Dumbledore", "I-PER"],
    [1, "founded", "O"],
    [1, "the", "O"],
    [1, "Order", "B-ORG"],
    [1, "of", "I-ORG"],
    [1, "the", "I-ORG"],
    [1, "Phoenix", "I-ORG"],
]
train_data = pd.DataFrame(
    train_data, columns=["sentence_id", "words", "labels"]
)

eval_data = [
    [0, "Sirius", "B-PER"],
    [0, "Black", "I-PER"],
    [0, "was", "O"],
    [0, "a", "O"],
    [0, "prisoner", "B-MISC"],
    [0, "at", "O"],
    [0, "Azkaban", "B-LOC"],
    [1, "Lord", "B-PER"],
    [1, "Voldemort", "I-PER"],
    [1, "founded", "O"],
    [1, "the", "O"],
    [1, "Death", "B-ORG"],
    [1, "Eaters", "I-ORG"],
]
eval_data = pd.DataFrame(
    eval_data, columns=["sentence_id", "words", "labels"]
)

In [3]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 50
model_args.overwrite_output_dir = True
model_args.train_batch_size = 16
model_args.evaluate_during_training = True

model = NERModel(
    "roberta", "outputs/best_model", args=model_args
)


In [ ]:
# Train the model
model.train_model(train_data, eval_data=eval_data)

In [6]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(eval_data)

In [7]:
result

{'eval_loss': 0.44025832414627075,
 'precision': 1.0,
 'recall': 0.8,
 'f1_score': 0.888888888888889}

In [8]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["Hermione was the best in her class"])

In [9]:
predictions

[[{'Hermione': 'B-PER'},
  {'was': 'O'},
  {'the': 'O'},
  {'best': 'O'},
  {'in': 'O'},
  {'her': 'O'},
  {'class': 'I-ORG'}]]

## Loading the data

In [34]:
MAX_SAMPLE = 20000

In [35]:
language_per_paper = pd.read_csv("Data analysis/colleridge_languages.csv", names=["paper_id", "lang"], header=0)

In [36]:
non_en_papers = set(language_per_paper[language_per_paper['lang'] != 'en']['paper_id'].values)

In [37]:
train_path = 'Data analysis/train.csv'
train = pd.read_csv(train_path)
train = train[:MAX_SAMPLE]

In [38]:
labels = train['cleaned_label'].unique()
labels

array(['national education longitudinal study', 'noaa tidal station',
       'slosh model', 'noaa c cap', 'aging integrated database agid ',
       'alzheimers disease neuroimaging initiative',
       'aging integrated database',
       'noaa national water level observation network',
       'noaa water level station',
       'baltimore longitudinal study of aging blsa ',
       'national water level observation network',
       'arms farm financial and crop production practices',
       'beginning postsecondary student',
       'noaa sea lake and overland surges from hurricanes',
       'noaa tide gauge',
       'the national institute on aging genetics of alzheimer s disease data storage site',
       'national center for education statistics common core of data',
       'national science foundation survey of industrial research and development',
       'baccalaureate and beyond',
       'noaa international best track archive for climate stewardship',
       'agricultural resource ma

In [52]:
paper_train_folder = 'Data analysis/train'
papers = {}
for paper_id in train['Id'].unique():
    with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        if paper_id in non_en_papers:
            continue
        papers[paper_id] = paper

                                  paper_id lang
6182  6ec6821b-9a23-48b6-be71-46e656603793   de
                                  paper_id lang
8258  94196698-d219-4158-9d96-fa078a8346a9   de
                                   paper_id lang
12586  e188b34a-73ee-456b-b10c-95aba94d3eb6   de
                                  paper_id lang
6005  6bd3c041-d51d-4d30-9c47-0bcd0c0a16ee   de
                                  paper_id lang
6273  7074f121-0761-4a56-b3da-09f2af2432a3   de
                                  paper_id lang
6568  75c66568-ae22-42de-9570-46b68c23d3fd   de
                                  paper_id lang
7356  8432e134-5cb0-4dfc-8794-b5b05ac8dea3   ca
                                  paper_id lang
9842  b0b545ad-42a9-48e7-bb86-32e5b9fca301   cy
                                  paper_id lang
3844  45f31d12-666d-4b9a-8c4e-4ce6e2c004b5   de
                                   paper_id lang
13525  f25fc3cd-f2db-405d-83c4-297c9c2c146d   de
                                  pa

                                  paper_id lang
9798  afd8938b-ff1b-4ea0-889c-7250a2259080   ro


In [40]:
sample_submission_path = 'Data analysis/sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_path)

test_papers = {}
paper_test_folder = 'Data analysis/test'
for paper_id in sample_submission['Id']:
    with open(f'{paper_test_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        test_papers[paper_id] = paper

In [41]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

def shorten_sentences(sentences):
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

### Train data

In [42]:
columns = ["sentence_id", "words", "labels"]
train_papers = pd.DataFrame(columns=columns)


In [43]:
import sys
from pprint import pprint

In [44]:
import itertools

In [45]:
dict(itertools.islice(papers.items(), 438, 439))

{'8fae6b1d-7e2e-4415-8ffd-2a03d150613e': [{'section_title': 'Abstract',
   'text': "Abstract. Alzheimer's disease (AD) is a degenerative brain disease that affects millions of people around the world. As populations in the United States and worldwide age, the prevalence of Alzheimer's disease will only increase. In turn, the social and financial costs of AD will create a difficult environment for many families and caregivers across the globe. By combining genetic information, brain scans, and clinical data, gathered over time through the Alzheimer's Disease Neuroimaging Initiative (ADNI), we propose a new Joint High-Order Multi-Modal Multi-Task Feature Learning method to predict the cognitive performance and diagnosis of patients with and without AD."},
  {'section_title': 'Introduction',
   'text': "Alzheimer's disease (AD) is a neurodegenerative condition characterized by the progressive loss of memory and cognitive functions. The Alzheimer's Association H. Wang-To whom all correspon

In [58]:
sentence_id = 0
with open("conll_formatted_annotated_training_corpus.txt", "a") as f:
    for paper_id, paper in tqdm(dict(itertools.islice(papers.items(), 5537, 14230)).items()):
        for section in tqdm(paper, leave=False):
            for sentence_id, sentence in enumerate(section['text'].split(". ")):
                sentence = clean_text(sentence)
                words = sentence.strip().split()
                sentence_length = len(words)
                tokens = ["O"] * sentence_length
                for label in train.loc[train['Id'] == paper_id]['cleaned_label'].values:
                    splitted = sentence.split(label)
                    if len(splitted) == 2:
                        left, right = splitted
                        label_index = len(left.strip().split())
                        if label_index < sentence_length and not tokens[label_index] in {'B-DS', 'I-DS'}:
                            tokens[label_index] = "B-DS"
                            label_length = len(label.split())
                            tokens[label_index+1:label_index+label_length] = ["I-DS"] * (label_length - 1)
                    if len(splitted) == 3:
                        left, middle, right = splitted
                        label_index = len(left.strip().split())                        
                        if label_index < sentence_length and not tokens[label_index] in {'B-DS', 'I-DS'}:
                            tokens[label_index] = "B-DS"
                            label_length = len(label.split())
                            tokens[label_index+1:label_index+label_length] = ["I-DS"] * (label_length - 1)

                        if label_index < sentence_length and not tokens[label_index] in {'B-DS', 'I-DS'}:
                            label_index = len(middle.strip().split())
                            tokens[label_index] = "B-DS"
                            tokens[label_index+1:label_index+label_length] = ["I-DS"] * (label_length - 1)
                for i in range(sentence_length):
                    f.write(f"{words[i]} {tokens[i]}\n")
                f.write("\n")

In [63]:
import random

def split_huge_file(file,out1,out2,percentage=0.75,seed=123):
    """Splits a file in 2 given the approximate `percentage` to go in the large file."""
    random.seed(seed)
    with open(file, 'r',encoding="utf-8") as fin, \
         open(out1, 'w') as foutBig, \
         open(out2, 'w') as foutSmall:
        
        r = random.random() 
        for line in fin:
            if line == '\n':
                r = random.random() 
            if r < percentage:
                foutBig.write(line)
            else:
                foutSmall.write(line)

In [64]:
split_huge_file('conll_formatted_annotated_training_corpus.txt', 'conll_formatted_annotated_training_corpus_train_0.8.txt', 'conll_formatted_annotated_training_corpus_eval_0.2.txt', percentage=0.8)

### Test data

In [27]:
test_rows = [] # test data in NER format
paper_length = [] # store the number of sentences each paper has

for paper_id in sample_submission['Id']:
    # load paper
    paper = papers[paper_id]
    
    # extract sentences
    sentences = [clean_training_text(sentence) for section in paper for sentence in section['text'].split('.')]
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    sentences = [sentence for sentence in sentences if any(word in sentence.lower() for word in ['data', 'study'])]
        
    # track which sentence belongs to which data point
    paper_length.append(len(sentences))
    
print(f'total number of sentences: {len(sentences)} \n small sample: {sentences[0]}')

total number of sentences: 84 
 small sample: This study contributes to the literature by exploring where people buy food and their purchases of specific foods food categories using three datasets varying in scope and detail


## Training model

In [3]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 10
model_args.overwrite_output_dir = True
model_args.train_batch_size = 8
model_args.lazy_loading = False
model_args.max_seq_length = 128
model_args.reprocess_input_data = True
model_args.evaluate_during_training = True

rob_ner = NERModel(
    "roberta", "roberta-base", args=model_args, labels=["O", "B-DS", "I-DS"]
)


loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\Thijs Gelton/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version"

In [ ]:
# Train the model
rob_ner.train_model(train_data='conll_formatted_annotated_training_corpus_train_0.8.txt', eval_data='conll_formatted_annotated_training_corpus_train_0.2.txt')

INFO:simpletransformers.ner.ner_model: Converting to features started.


In [6]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(eval_data)

In [7]:
result

{'eval_loss': 0.44025832414627075,
 'precision': 1.0,
 'recall': 0.8,
 'f1_score': 0.888888888888889}

In [184]:
" ".join(list(map(lambda x: x['text'], next(iter(test_papers.items()))[1])))

'Cognitive deficits and reduced educational achievement are common in psychiatric illness; understanding the genetic basis of cognitive and educational deficits may be informative about the etiology of psychiatric disorders. A recent, large genomewide association study (GWAS) reported a genome-wide significant locus for years of education, which subsequently demonstrated association to general cognitive ability ("g") in overlapping cohorts. The current study was designed to test whether GWAS hits for educational attainment are involved in general cognitive ability in an independent, large-scale collection of cohorts. Using cohorts in the Cognitive Genomics Consortium (COGENT; up to 20,495 healthy individuals), we examined the relationship between g and variants associated with educational attainment. We next conducted meta-analyses with 24,189 individuals with neurocognitive data from the educational attainment studies, and then with 53,188 largely independent individuals from a recent

In [185]:
# Make predictions with the model
predictions, raw_outputs = rob_ner.predict([" ".join(list(map(lambda x: x['text'], next(iter(test_papers.items()))[1])))])

INFO:simpletransformers.ner.ner_model: Converting to features started.


In [190]:
predictions

[[{'Cognitive': 'O'},
  {'deficits': 'O'},
  {'and': 'O'},
  {'reduced': 'O'},
  {'educational': 'O'},
  {'achievement': 'O'},
  {'are': 'O'},
  {'common': 'O'},
  {'in': 'O'},
  {'psychiatric': 'O'},
  {'illness;': 'O'},
  {'understanding': 'O'},
  {'the': 'O'},
  {'genetic': 'O'},
  {'basis': 'O'},
  {'of': 'O'},
  {'cognitive': 'O'},
  {'and': 'O'},
  {'educational': 'O'},
  {'deficits': 'O'},
  {'may': 'O'},
  {'be': 'O'},
  {'informative': 'O'},
  {'about': 'O'},
  {'the': 'O'},
  {'etiology': 'O'},
  {'of': 'O'},
  {'psychiatric': 'O'},
  {'disorders.': 'O'},
  {'A': 'O'},
  {'recent,': 'O'},
  {'large': 'O'},
  {'genomewide': 'O'},
  {'association': 'O'},
  {'study': 'O'},
  {'(GWAS)': 'O'},
  {'reported': 'O'},
  {'a': 'O'},
  {'genome-wide': 'O'},
  {'significant': 'O'},
  {'locus': 'O'},
  {'for': 'O'},
  {'years': 'O'},
  {'of': 'O'},
  {'education,': 'O'},
  {'which': 'O'},
  {'subsequently': 'O'},
  {'demonstrated': 'O'},
  {'association': 'O'},
  {'to': 'O'},
  {'general'